In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_excel('../Base_de_datos/Productos.xlsx')
df

,id_producto,nombre_producto,categoria,precio_unitario
0,1,Coca Cola 1.5L,Alimentos,2347
1,2,Pepsi 1.5L,Limpieza,4973
2,3,Sprite 1.5L,Alimentos,4964
3,4,Fanta Naranja 1.5L,Limpieza,2033
4,5,Agua Mineral 500ml,Alimentos,4777
...,...,...,...,...
95,96,Suavizante 1L,Limpieza,4920
96,97,Limpiavidrios 500ml,Alimentos,872
97,98,Desengrasante 500ml,Limpieza,2843
98,99,Esponjas x3,Alimentos,2430


# **1. Analizar problemas con el dataset**

### **Tipos de datos y valores nulos**

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_producto      100 non-null    int64 
 1   nombre_producto  100 non-null    object
 2   categoria        100 non-null    object
 3   precio_unitario  100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.3+ KB


### **Estadísticas generales**

In [25]:
df.describe(include='all')

,id_producto,nombre_producto,categoria,precio_unitario
count,100.000000,100,100,100.000000
unique,NaN,100,2,NaN
top,NaN,Coca Cola 1.5L,Alimentos,NaN
freq,NaN,1,50,NaN
mean,50.500000,NaN,NaN,2718.550000
std,29.011492,NaN,NaN,1381.635324
min,1.000000,NaN,NaN,272.000000
25%,25.750000,NaN,NaN,1590.000000
50%,50.500000,NaN,NaN,2516.000000
75%,75.250000,NaN,NaN,4026.500000


### **Conteo de valores faltantes**

In [26]:
df.isnull().sum()

id_producto        0
nombre_producto    0
categoria          0
precio_unitario    0
dtype: int64

### **Conteo de filas duplicadas**

In [27]:
df.duplicated().sum()

np.int64(0)

### **Frecuencia por categoría**

In [28]:
df['categoria'].value_counts()

categoria
Alimentos    50
Limpieza     50
Name: count, dtype: int64

### **Verificación de valores únicos en columnas categóricas**

In [29]:
df['categoria'].unique()

array(['Alimentos', 'Limpieza'], dtype=object)

### **Verificación de rangos de precios**

In [30]:
(df['precio_unitario'] > 0).sum()

np.int64(100)

# **2. Limpieza de datos**

### **Eliminación de filas duplicadas**

In [31]:
df = df.drop_duplicates()

### **Quitar espacios y capitalizar**

In [32]:
df['categoria'] = df['categoria'].str.strip().str.title()

### **Eliminar filas con valores nulos en 'id_producto' o 'nombre_producto'**

In [33]:
df = df.dropna(subset=['id_producto', 'nombre_producto'])

# **Estandarización**

### **Asegurar que todos los datos sigan el mismo formato:**
- Nombres de productos con mayúscula inicial
- Categorías estandarizadas  
- Precios con formato numérico correcto

In [34]:
df['nombre_producto'] = df['nombre_producto'].str.title()
df['precio_unitario'] = pd.to_numeric(df['precio_unitario'], errors='coerce')
df['categoria'] = df['categoria'].replace({
    'Electronica': 'Electrónica',
    'Electrodomesticos': 'Electrodomésticos',
    'Hogar Y Jardin': 'Hogar y Jardín'
})

# **3. Normalización de datos**

### **Normalización de IDs de producto**

In [35]:
df['id_producto'] = pd.to_numeric(df['id_producto'], errors='coerce')

### **Normalización de nombres de producto**

In [36]:
df['nombre_producto'] = df['nombre_producto'].str.strip().str.replace(r'\s+', ' ', regex=True)

### **Normalización de precios**

In [37]:
df['precio_unitario'] = df['precio_unitario'].abs()
df.drop_duplicates(subset=['nombre_producto'], inplace=True)

### **Normalización de categorías (adicional)**

In [38]:
variaciones_adicionales = {
    'Tecnologia': 'Tecnología',
    'Ropa Y Accesorios': 'Ropa y Accesorios',
    'Deportes Y Aire Libre': 'Deportes y Aire Libre'
}
df['categoria'] = df['categoria'].replace(variaciones_adicionales)

### **One-hot encoding para variables categóricas**

In [39]:
df = pd.get_dummies(df, columns=['categoria'], prefix='cat')
df

,id_producto,nombre_producto,precio_unitario,cat_Alimentos,cat_Limpieza
0,1,Coca Cola 1.5L,2347,True,False
1,2,Pepsi 1.5L,4973,False,True
2,3,Sprite 1.5L,4964,True,False
3,4,Fanta Naranja 1.5L,2033,False,True
4,5,Agua Mineral 500Ml,4777,True,False
...,...,...,...,...,...
95,96,Suavizante 1L,4920,False,True
96,97,Limpiavidrios 500Ml,872,True,False
97,98,Desengrasante 500Ml,2843,False,True
98,99,Esponjas X3,2430,True,False


In [43]:
# Convertir las columnas de dummies a tipo entero

dummies_cols = ['cat_Alimentos', 'cat_Limpieza']
df[dummies_cols] = df[dummies_cols].astype(int)

df

,id_producto,nombre_producto,precio_unitario,cat_Alimentos,cat_Limpieza
0,1,Coca Cola 1.5L,2347,1,0
1,2,Pepsi 1.5L,4973,0,1
2,3,Sprite 1.5L,4964,1,0
3,4,Fanta Naranja 1.5L,2033,0,1
4,5,Agua Mineral 500Ml,4777,1,0
...,...,...,...,...,...
95,96,Suavizante 1L,4920,0,1
96,97,Limpiavidrios 500Ml,872,1,0
97,98,Desengrasante 500Ml,2843,0,1
98,99,Esponjas X3,2430,1,0


# **4. Validación Post-Limpieza y Normalización**

### **Verificar el resultado de la limpieza**

In [41]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_producto      100 non-null    int64 
 1   nombre_producto  100 non-null    object
 2   precio_unitario  100 non-null    int64 
 3   cat_Alimentos    100 non-null    int64 
 4   cat_Limpieza     100 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 4.0+ KB


,id_producto,nombre_producto,precio_unitario,cat_Alimentos,cat_Limpieza
0,1,Coca Cola 1.5L,2347,1,0
1,2,Pepsi 1.5L,4973,0,1
2,3,Sprite 1.5L,4964,1,0
3,4,Fanta Naranja 1.5L,2033,0,1
4,5,Agua Mineral 500Ml,4777,1,0


### **Comparación antes vs después**

# **5. Exportar datos limpios y normalizados**

In [42]:
df.to_csv('../Base_de_datos_limpia/Productos_limpio.csv', index=False)